In [147]:
from numba import cuda,jit
import numpy as np
import cv2
import math
from copy import copy

In [124]:
@cuda.jit(device=True)
def compare_pixel(pixela, pixelb):
    c=0
    for i in range(3):
        c+=(pixela[i]-pixelb[i])**2
    return c**0.5

In [125]:
@cuda.jit
def gcompare(imga, imgb, C):
    y, x = cuda.grid(2)
    if y>=imga.shape[0] or x>=imga.shape[1]:
        return
    
    C[y,x]=compare_pixel(imga[y,x], imgb[y,x])

In [143]:
def compare(imga, imgb):
    assert imga.shape==imgb.shape
    
    tpb = 16
    bpgy = (imga.shape[0]-1)//tpb+1
    bpgx = (imga.shape[1]-1)//tpb+1
    
    C=np.array(np.zeros(imga.shape[0:2]), dtype=np.float32)

    gcompare[(bpgy, bpgx), (tpb, tpb)](dimga, dimgb, dC)
    dC.copy_to_host()
    return np.sum(C)


In [144]:
img=cv2.imread("puzzle.jpg", 1)
imga=img[0:300,0:300]
imgb=img[100:400,100:400]
imgb+=1
array=np.ones((16))

In [145]:
%timeit compare(imga, imgb)

2.11 ms ± 99 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [146]:
def locate_one_piece(pieces, solution_pieces):
    pieces, solution_pieces = preprocess_pieces(pieces, solution_pieces)
    resemblance_array=np.empty((pieces.shape[0]))
    max=(0, (None, None))
    for i in range(pieces.shape[0]):
        match_size(pieces[i])
        max=resemblance_array[0]
        resemblance_array[i]=compare(pieces[i], solution_pieces[0][i])
    

In [88]:
def preprocess_pieces(pieces, solution_pieces)
    if not(isinstance(pieces, np.ndarray) and isinstance(solution_pieces[0], np.ndarray) and isinstance(solution_pieces[1], np.ndarray)) raise TypeError("Wrong object type.")
    if not(len(pieces.shape)==4 and len(solution_pieces[0].shape)==4): raise Exception("Incorrect array shape.")
    if pieces.shape[0] != solution_pieces[0].shape[0]: raise Exception("Number of pieces don't match!")
    
    if pieces.shape != solution_pieces[0].shape
        for i in range(pieces.shape[0])